# Factor-mimicking ETFs

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
base_data = pd.read_sas("CA.sas7bdat", encoding = 'latin-1')
base_data['yyyymm'] = base_data['date'].dt.year*100 + base_data['date'].dt.month

In [5]:
base_data.head()

,permno,date,ret,ret_t1,TICKER,COMNAM,PRC,SHROUT,datadate,fyearq,...,ep1_winsorized_zscore,ep2_winsorized_zscore,gvkey,beta,ivol,mom,beta_winsorized,ivol_winsorized,mom_winsorized,yyyymm
0,10107.0,1997-01-31,0.234493,-0.044110,MSFT,MICROSOFT CORP,102.0000,1198000.0,1996-09-30,1997.0,...,-0.066718,0.274438,012141,NaN,0.012851,NaN,NaN,0.012851,NaN,199701
1,10107.0,1997-02-28,-0.044110,-0.059610,MSFT,MICROSOFT CORP,97.5000,1198000.0,1996-09-30,1997.0,...,0.001182,0.337490,012141,NaN,0.014933,NaN,NaN,0.014933,NaN,199702
2,10107.0,1997-03-31,-0.059610,0.325153,MSFT,MICROSOFT CORP,91.6875,1191000.0,1996-09-30,1997.0,...,-0.001393,0.309762,012141,NaN,0.016597,NaN,NaN,0.016597,NaN,199703
3,10107.0,1997-04-30,0.325153,0.020576,MSFT,MICROSOFT CORP,121.5000,1191000.0,1996-12-31,1997.0,...,0.003298,-0.021655,012141,NaN,0.022905,NaN,NaN,0.022905,NaN,199704
4,10107.0,1997-05-30,0.020576,0.019153,MSFT,MICROSOFT CORP,124.0000,1191000.0,1996-12-31,1997.0,...,-0.078655,-0.149326,012141,NaN,0.011146,NaN,NaN,0.011146,NaN,199705


### Quintile portfolios

In [6]:
def portfolio_sorting(df, factor):
    month_groups = df.groupby(["yyyymm"])
    
    dfs = []

    for name, group in month_groups:
        group.dropna(subset=[factor], inplace=True)
        if len(group) == 0:
            continue
        else:
            group['quintile'] = pd.qcut(group[factor], 5, labels=np.arange(1, 6, 1))
            dfs.append(group)
    
    output = pd.concat(dfs)
    output = output.sort_values(['permno', 'yyyymm'])
    output = output[(output['yyyymm'] >= 200001) & (output['yyyymm'] <= 202111)]
    
    return output

In [7]:
lnSize_portfolio = portfolio_sorting(base_data, 'lnSize_winsorized')
bk2mkt_portfolio = portfolio_sorting(base_data, 'bk2mkt_winsorized')
ep1_portfolio = portfolio_sorting(base_data, 'ep1_winsorized')
beta_portfolio = portfolio_sorting(base_data, 'beta_winsorized')
ivol_portfolio = portfolio_sorting(base_data, 'ivol_winsorized')
mom_portfolio = portfolio_sorting(base_data, 'mom_winsorized')

In [191]:
### output six quintile portfolios

lnSize_portfolio.to_excel("lnSize_quintile.xlsx")
bk2mkt_portfolio.to_excel("bk2mkt_quintile.xlsx")
ep1_portfolio.to_excel("ep1_quintile.xlsx")
beta_portfolio.to_excel("beta_quintile.xlsx")
ivol_portfolio.to_excel("ivol_quintile.xlsx")
mom_portfolio.to_excel("mom_quintile.xlsx")

In [8]:
ff_factors = pd.read_excel("ff_factors.xlsx")

ff_factors.rename({'Excess Return on the Market': 'MKTRF', 
                   'Small-Minus-Big Return': 'SMB',
                   'High-Minus-Low Return': 'HML',
                   'Risk-Free Return Rate (One Month Treasury Bill Rate)': 'RF',
                   'Momentum Factor': 'MOM',
                   'Date (SAS). Last Trading Day of the Month': 'Date'
                  }, 
                  axis=1, inplace=True)

ff_factors['yyyymm'] = ff_factors['Date'].dt.year*100 + ff_factors['Date'].dt.month

In [9]:
ff_factors.head()

,MKTRF,SMB,HML,RF,MOM,Date,yyyymm
0,0.0551,0.0162,0.0175,0.0080,0.0755,1980-01-31,198001
1,-0.0122,-0.0185,0.0061,0.0089,0.0788,1980-02-29,198002
2,-0.1290,-0.0664,-0.0101,0.0121,-0.0955,1980-03-31,198003
3,0.0397,0.0105,0.0106,0.0126,-0.0043,1980-04-30,198004
4,0.0526,0.0213,0.0038,0.0081,-0.0112,1980-05-30,198005


In [10]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from scipy import stats

In [11]:
def hedge_portfolio(df, long, short, factor):
    month_groups = df.groupby(["yyyymm"])
    months = []
    returns = []
    
    for name, group in month_groups:
        long_return = group[group['quintile'] == long]['ret'].mean()
        short_return = group[group['quintile'] == short]['ret'].mean()
        ret = long_return - short_return
        
        months.append(name)
        returns.append(ret)
        
    portfolio = {'yyyymm': months,
                 'Return': returns,
                }

    df_portfolio = pd.DataFrame(portfolio)
    
    df_portfolio = pd.merge(df_portfolio, ff_factors,
                  on='yyyymm', how='inner')
    
    df_portfolio['Return+1'] = df_portfolio['Return'] + 1
    df_portfolio['MKT+1'] = df_portfolio['MKTRF'] + df_portfolio['RF'] + 1
    
    overall_return = df_portfolio['Return+1'].product() - 1
    mkt_return = df_portfolio['MKT+1'].product() - 1
    excess_return = overall_return - mkt_return
    
    CAPMmodel = linear_model.LinearRegression().fit(df_portfolio[['MKTRF']], 
                                                    df_portfolio["Return"])
    ff3model = linear_model.LinearRegression().fit(df_portfolio[["MKTRF", "SMB", "HML", "MOM"]], 
                                                   df_portfolio["Return"])
    
    capm_alpha = CAPMmodel.intercept_
    ff_alpha = ff3model.intercept_
    
    sharpe_ratio = (df_portfolio['Return'] - df_portfolio['RF']).mean() / df_portfolio['Return'].std()
    
    hedge_stat = {'Factor': [factor],
                  'Overall Return': [overall_return],
                  'Excess Return over the Market': [excess_return],
                  'CAPM-alpha': [capm_alpha],
                  'Fama-French four-factor alpha': [ff_alpha],
                  'Sharpe Ratio': [sharpe_ratio]
                 }
    df_stat = pd.DataFrame(hedge_stat)
    df_stat.set_index('Factor', inplace=True)
    
    return df_stat

In [12]:
lnSize_portfolio = portfolio_sorting(base_data, 'lnSize_winsorized')
bk2mkt_portfolio = portfolio_sorting(base_data, 'bk2mkt_winsorized')
ep1_portfolio = portfolio_sorting(base_data, 'ep1_winsorized')
beta_portfolio = portfolio_sorting(base_data, 'beta_winsorized')
ivol_portfolio = portfolio_sorting(base_data, 'ivol_winsorized')
mom_portfolio = portfolio_sorting(base_data, 'mom_winsorized')

In [34]:
hedge_portfolio(lnSize_portfolio, 1, 5, 'lnSize')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
lnSize,4.121682,0.054397,0.006068,0.005968,0.106597


In [184]:
hedge_portfolio(bk2mkt_portfolio, 5, 1, 'bk2mkt')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
bk2mkt,-0.999876,-5.067161,-0.030425,-0.02958,-0.561076


In [36]:
hedge_portfolio(ep1_portfolio, 1, 5, 'ep1')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
ep1,230.573491,226.506205,0.020222,0.02052,0.301772


In [37]:
hedge_portfolio(beta_portfolio, 5, 1, 'beta')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
beta,0.257883,-3.809402,-0.003963,-0.002269,0.048446


In [38]:
hedge_portfolio(ivol_portfolio, 5, 1, 'ivol')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
ivol,153.33002,149.262735,0.017608,0.019566,0.231234


In [39]:
hedge_portfolio(mom_portfolio, 5, 1, 'mom')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
mom,2.979028e+07,2.979028e+07,0.073747,0.068535,0.789939


### Question 2 Betting against beta strategy

In [40]:
hedge_portfolio(beta_portfolio, 1, 5, 'beta')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
beta,-0.960299,-5.027584,0.003963,0.002269,-0.072232


### Factor-mimicking ETF

In [67]:
month_groups = ivol_portfolio.groupby(["yyyymm"])
months = []
returns = []
stocks = []

for name, group in month_groups:
    long_return = group[group['quintile'] == 5]['ret'].mean()

    months.append(name)
    returns.append(long_return)
    stocks.append(group[group['quintile'] == 5]['permno'].unique())

portfolio = {'yyyymm': months,
             'Return': returns,
            }

df_portfolio = pd.DataFrame(portfolio)

df_portfolio = pd.merge(df_portfolio, ff_factors,
              on='yyyymm', how='inner')

df_portfolio['Return+1'] = df_portfolio['Return'] + 1
df_portfolio['MKT+1'] = df_portfolio['MKTRF'] + df_portfolio['RF'] + 1

overall_return = df_portfolio['Return+1'].product() - 1
mkt_return = df_portfolio['MKT+1'].product() - 1
excess_return = overall_return - mkt_return

ff3model = linear_model.LinearRegression().fit(df_portfolio[["MKTRF", "SMB", "HML", "MOM"]], 
                                               df_portfolio["Return"])

ff_alpha = ff3model.intercept_

hedge_stat = {'Factor': ['ivol'],
              'Excess Return over the Market': [excess_return],
              'Fama-French four-factor alpha': [ff_alpha]
             }
ivol_stat = pd.DataFrame(hedge_stat)
ivol_stat.set_index('Factor', inplace=True)

ivol_stat

,Excess Return over the Market,Fama-French four-factor alpha
Factor,,
ivol,2693.613201,0.027981


In [57]:
turnovers = []
for i in range(1, len(stocks)):
    current = stocks[i]
    last = stocks[i-1]
    total = len(current)
    count = 0
    for stock in last:
        if stock in current:
            count = count + 1
    turnover = count / total
    turnovers.append(turnover)

annual_turnover = sum(turnovers) / len(turnovers) * 12
annual_turnover

5.78016274251119

### Multi-factor ETF

In [68]:
month_groups = ivol_portfolio.groupby(["yyyymm"])
months = []
ivol_returns = []

for name, group in month_groups:
    long_return = group[group['quintile'] == 5]['ret'].mean()
    short_return = group[group['quintile'] == 1]['ret'].mean()
    ret = long_return - short_return

    months.append(name)
    ivol_returns.append(ret)

    
month_groups = beta_portfolio.groupby(["yyyymm"])
beta_returns = []

for name, group in month_groups:
    long_return = group[group['quintile'] == 1]['ret'].mean()
    short_return = group[group['quintile'] == 5]['ret'].mean()
    ret = long_return - short_return

    beta_returns.append(ret)
    

portfolio = {'yyyymm': months,
             'Beta Return': beta_returns,
             'Ivol Return': ivol_returns
            }

df_portfolio = pd.DataFrame(portfolio)

df_portfolio = pd.merge(df_portfolio, ff_factors,
              on='yyyymm', how='inner')

df_portfolio['Return'] = (df_portfolio['Beta Return'] + df_portfolio['Ivol Return']) / 2

ff3model = linear_model.LinearRegression().fit(df_portfolio[["MKTRF", "SMB", "HML", "MOM"]], 
                                               df_portfolio["Return"])

ff_alpha = ff3model.intercept_



sharpe_ratio = (df_portfolio['Return'] - df_portfolio['RF']).mean() / df_portfolio['Return'].std()

print('FF-4 alpha of the ETF: ', round(ff_alpha, 4))
print('Sharpe ratio of the ETF: ', round(sharpe_ratio, 4))

FF-4 alpha of the ETF:  0.0109
Sharpe ratio of the ETF:  0.2221


In [185]:
df_portfolio['Return after paying fee'] = df_portfolio['Return'] - 0.0005

ff3model = linear_model.LinearRegression().fit(df_portfolio[["MKTRF", "SMB", "HML", "MOM"]], 
                                               df_portfolio["Return after paying fee"])

ff_alpha = ff3model.intercept_


sharpe_ratio = (df_portfolio['Return after paying fee'] - df_portfolio['RF']).mean() / df_portfolio['Return after paying fee'].std()

print('FF-4 alpha of the ETF: ', round(ff_alpha, 4))
print('Sharpe ratio of the ETF: ', round(sharpe_ratio, 4))

FF-4 alpha of the ETF:  0.0104
Sharpe ratio of the ETF:  0.2072


In [106]:
df_portfolio

,yyyymm,Beta Return,Ivol Return,MKTRF,SMB,HML,RF,MOM,Date,Return,Return after paying fee
0,200001,0.072979,0.037417,-0.0474,0.0577,-0.0188,0.0041,0.0192,2000-01-31,0.055198,0.049198
1,200002,-0.455238,0.592424,0.0245,0.2136,-0.0959,0.0043,0.1820,2000-02-29,0.068593,0.062593
2,200003,0.010277,-0.023956,0.0520,-0.1720,0.0813,0.0047,-0.0683,2000-03-31,-0.006840,-0.012840
3,200004,0.190327,-0.208600,-0.0640,-0.0668,0.0726,0.0046,-0.0839,2000-04-28,-0.009137,-0.015137
4,200005,0.192129,-0.029104,-0.0442,-0.0605,0.0475,0.0050,-0.0898,2000-05-31,0.081513,0.075513
...,...,...,...,...,...,...,...,...,...,...,...
258,202107,-0.042019,-0.041390,0.0127,-0.0397,-0.0178,0.0000,-0.0228,2021-07-30,-0.041704,-0.047704
259,202108,-0.048765,-0.009535,0.0291,-0.0042,-0.0015,0.0000,0.0245,2021-08-31,-0.029150,-0.035150
260,202109,-0.017572,0.006187,-0.0437,0.0071,0.0508,0.0000,0.0149,2021-09-30,-0.005692,-0.011692
261,202110,-0.035453,-0.031308,0.0665,-0.0235,-0.0049,0.0000,0.0319,2021-10-29,-0.033380,-0.039380


### Apply to Larger Samples

In [87]:
all_data = pd.read_sas("all.sas7bdat", encoding = 'latin-1')
all_data['yyyymm'] = all_data['DATE'].dt.year*100 + all_data['DATE'].dt.month

In [88]:
all_data.head()

,PERMNO,DATE,SHRCD,EXCHCD,PRC,RET,SHROUT,yyyymm
0,10001.0,1999-01-29,11.0,3.0,9.6250,0.000000,2421.0,199901
1,10001.0,1999-02-26,11.0,3.0,8.7500,-0.090909,2421.0,199902
2,10001.0,1999-03-31,11.0,3.0,8.7500,0.013143,2421.0,199903
3,10001.0,1999-04-30,11.0,3.0,8.8750,0.014286,2421.0,199904
4,10001.0,1999-05-28,11.0,3.0,-8.6875,-0.021127,2421.0,199905


In [89]:
all_data = all_data[all_data['PRC'] > 0]
all_data.dropna(subset=['PRC'], inplace=True)
all_data.dropna(subset=['SHROUT'], inplace=True)

all_data['Market Value'] = all_data['PRC'] * all_data['SHROUT'] * 1000
all_data['lnSize'] = np.log(all_data['Market Value'])

In [91]:
all_data.head()

,PERMNO,DATE,SHRCD,EXCHCD,PRC,RET,SHROUT,yyyymm,Market Value,lnSize
0,10001.0,1999-01-29,11.0,3.0,9.625,0.000000,2421.0,199901,23302125.0,16.964055
1,10001.0,1999-02-26,11.0,3.0,8.750,-0.090909,2421.0,199902,21183750.0,16.868745
2,10001.0,1999-03-31,11.0,3.0,8.750,0.013143,2421.0,199903,21183750.0,16.868745
3,10001.0,1999-04-30,11.0,3.0,8.875,0.014286,2421.0,199904,21486375.0,16.882930
5,10001.0,1999-06-30,11.0,3.0,8.625,0.006619,2450.0,199906,21131250.0,16.866264


#### a. all firms in NYSE/Nasdaq

In [97]:
all_data.rename({'PERMNO': 'permno', 
                 'RET': 'ret'}, 
                  axis=1, inplace=True)
all_portfolio = portfolio_sorting(all_data, 'lnSize')

In [98]:
all_portfolio.head()

,permno,DATE,SHRCD,EXCHCD,PRC,ret,SHROUT,yyyymm,Market Value,lnSize,quintile
12,10001.0,2000-01-31,11.0,3.0,8.125,-0.044118,2450.0,200001,19906250.0,16.806544,1
13,10001.0,2000-02-29,11.0,3.0,8.250,0.015385,2450.0,200002,20212500.0,16.821812,1
17,10001.0,2000-06-30,11.0,3.0,8.000,0.027668,2475.0,200006,19800000.0,16.801192,1
18,10001.0,2000-07-31,11.0,3.0,7.875,-0.015625,2475.0,200007,19490625.0,16.785444,1
19,10001.0,2000-08-31,11.0,3.0,8.250,0.047619,2475.0,200008,20418750.0,16.831964,1


In [107]:
hedge_portfolio(all_portfolio, 1, 5, 'lnSize')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
lnSize,-0.999157,-5.066442,-0.026342,-0.025066,-0.486051


#### b. micro-caps removed

In [121]:
micro_removed_data = all_portfolio[all_portfolio['quintile'] >= 4]
micro_removed_data = micro_removed_data.drop(columns=['quintile'])
micro_removed_data.head()

,permno,DATE,SHRCD,EXCHCD,PRC,ret,SHROUT,yyyymm,Market Value,lnSize
425,10016.0,2000-10-31,11.0,3.0,23.0000,0.298060,16947.0,200010,389781000.0,19.781096
426,10016.0,2000-11-30,11.0,3.0,19.9375,-0.133152,16947.0,200011,337880812.5,19.638204
427,10016.0,2000-12-29,11.0,3.0,22.3750,0.122257,16964.0,200012,379569500.0,19.754548
428,10016.0,2001-01-31,11.0,3.0,23.0625,0.030726,16964.0,200101,391232250.0,19.784812
429,10016.0,2001-02-28,11.0,3.0,22.7500,-0.013550,16964.0,200102,385931000.0,19.771169


In [122]:
micro_removed_portfolio = portfolio_sorting(micro_removed_data, 'lnSize')

In [124]:
micro_removed_portfolio.head()

,permno,DATE,SHRCD,EXCHCD,PRC,ret,SHROUT,yyyymm,Market Value,lnSize,quintile
425,10016.0,2000-10-31,11.0,3.0,23.0000,0.298060,16947.0,200010,389781000.0,19.781096,1
426,10016.0,2000-11-30,11.0,3.0,19.9375,-0.133152,16947.0,200011,337880812.5,19.638204,1
427,10016.0,2000-12-29,11.0,3.0,22.3750,0.122257,16964.0,200012,379569500.0,19.754548,1
428,10016.0,2001-01-31,11.0,3.0,23.0625,0.030726,16964.0,200101,391232250.0,19.784812,1
429,10016.0,2001-02-28,11.0,3.0,22.7500,-0.013550,16964.0,200102,385931000.0,19.771169,1


In [125]:
hedge_portfolio(micro_removed_portfolio, 1, 5, 'lnSize')

,Overall Return,Excess Return over the Market,CAPM-alpha,Fama-French four-factor alpha,Sharpe Ratio
Factor,,,,,
lnSize,1.010089,-3.057197,0.00122,0.00067,0.061192
